<a href="https://colab.research.google.com/github/bimarshak7/nepali_text_summarizer/blob/main/create_Word2VecEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load dataset

In [2]:
import pandas as pd

In [52]:
news2  = pd.read_csv("/content/drive/MyDrive/NLP/ap2.csv")
news2.head()

,id,date_published,title,news
0,208422p1,"असार ३०, २०७९ बिहिबार ११:०:०",सरकारप्रतिको गिर्दो जनविश्वास,जनताको विश्वास र बैधतामा शासन गर्ने सरकारहरू ज...
1,159079p1,"असार ५, २०७७ शुक्रबार १२:४८:०",लद्दाखमा चीनको लक्ष्य,अहिले लद्दाख संकटलाई लिएर भारतमा भएको बहस एउटै...
2,157801p1,"जेठ २३, २०७७ शुक्रबार ७:०:०",आगामी दिनमा बसाल्नुपर्ने आनीबानी,पहिलाको जीवन सरल थियो। न मुखमा मास्क चाहिन्थ्य...
3,157795p1,"जेठ २३, २०७७ शुक्रबार ५:५९:०",आधुनिक अन्धकार युगमा पिरती,जन्मनेबित्तिकै मुहारमा जातीयताको चिह्न लगाइद...
4,157001p1,"जेठ १४, २०७७ बुधबार १३:३२:०",शक्तिशाली देशको कोरोनाविरुद्ध कमजोर कदम,"अमेरिकाका डोनाल्ड ट्रम्प, रुसका भ्लादिमिर पुटि..."


In [53]:
news3 = pd.read_csv("/content/drive/MyDrive/NLP/NLP/news_data.csv",delimiter="~#")
news3 = news3.drop_duplicates(keep='first', inplace=False)
news3.head()

,id,date_published,title,news
0,23,2013-03-31,सरकारको सेन्सरशाला,प्रायः विवादका विषयमा विवादै चल्ने गरी चलचित...
1,24,2013-03-31,अब निशानामा न्यायपालिका,सिद्धान्ततः एकीकृत नेपाल कम्युनिष्ट पार्टी–म...
2,25,2013-03-31,बग्दैछ आइसीको उल्टो गंगा,"वीरगन्ज, चैत १८– बिरामी नातिको माया कतिसम्म ..."
3,26,2013-03-31,डिल्ले दमाइका शोक धुन,चैनपुर (संखुवासभा)– २०१३ सालताका पूर्वी नेपा...
4,27,2013-04-01,मंसिरमा चुनावः खनाल,"काठमाडौं, चैत १८ (सेतोपाटी)– एमाले अध्यक्ष झ..."


In [54]:
newsMerge = pd.concat([news2,news3])
newsMerge.head(2)

,id,date_published,title,news
0,208422p1,"असार ३०, २०७९ बिहिबार ११:०:०",सरकारप्रतिको गिर्दो जनविश्वास,जनताको विश्वास र बैधतामा शासन गर्ने सरकारहरू ज...
1,159079p1,"असार ५, २०७७ शुक्रबार १२:४८:०",लद्दाखमा चीनको लक्ष्य,अहिले लद्दाख संकटलाई लिएर भारतमा भएको बहस एउटै...


In [56]:
newsMerge.isna().sum()

id                  0
date_published    516
title             516
news              516
dtype: int64

In [57]:
newsMerge.dropna(inplace=True)

In [58]:
newsMerge.isna().sum()

id                0
date_published    0
title             0
news              0
dtype: int64

In [59]:
newsMerge['title'] = newsMerge['title'].apply(lambda x: x+'।') # add purnabiram at end of news title

In [60]:
newsMerge.to_csv('/content/drive/MyDrive/NLP/news_big.csv', sep=",", index=False) # save merged dataset for later

In [61]:
newsMerge[['title','news']].to_csv('/content/drive/MyDrive/NLP/corpus.txt', sep="\n", index=False,header=False) # save news title and content as text corpus

In [62]:
!du -h /content/drive/MyDrive/NLP/corpus.txt #file size of text corpus

1.2G	/content/drive/MyDrive/NLP/corpus.txt


## Preprocess text

In [3]:
from nltk.tokenize import word_tokenize
import string
import warnings
warnings.filterwarnings(action = 'ignore')

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stop_words = stopwords.words('nepali')
print("Stopwords: ",stop_words)

Stopwords:  ['छ', 'र', 'पनि', 'छन्', 'लागि', 'भएको', 'गरेको', 'भने', 'गर्न', 'गर्ने', 'हो', 'तथा', 'यो', 'रहेको', 'उनले', 'थियो', 'हुने', 'गरेका', 'थिए', 'गर्दै', 'तर', 'नै', 'को', 'मा', 'हुन्', 'भन्ने', 'हुन', 'गरी', 'त', 'हुन्छ', 'अब', 'के', 'रहेका', 'गरेर', 'छैन', 'दिए', 'भए', 'यस', 'ले', 'गर्नु', 'औं', 'सो', 'त्यो', 'कि', 'जुन', 'यी', 'का', 'गरि', 'ती', 'न', 'छु', 'छौं', 'लाई', 'नि', 'उप', 'अक्सर', 'आदि', 'कसरी', 'क्रमशः', 'चाले', 'अगाडी', 'अझै', 'अनुसार', 'अन्तर्गत', 'अन्य', 'अन्यत्र', 'अन्यथा', 'अरु', 'अरुलाई', 'अर्को', 'अर्थात', 'अर्थात्', 'अलग', 'आए', 'आजको', 'ओठ', 'आत्म', 'आफू', 'आफूलाई', 'आफ्नै', 'आफ्नो', 'आयो', 'उदाहरण', 'उनको', 'उहालाई', 'एउटै', 'एक', 'एकदम', 'कतै', 'कम से कम', 'कसै', 'कसैले', 'कहाँबाट', 'कहिलेकाहीं', 'का', 'किन', 'किनभने', 'कुनै', 'कुरा', 'कृपया', 'केही', 'कोही', 'गए', 'गरौं', 'गर्छ', 'गर्छु', 'गर्नुपर्छ', 'गयौ', 'गैर', 'चार', 'चाहनुहुन्छ', 'चाहन्छु', 'चाहिए', 'छू', 'जताततै', 'जब', 'जबकि', 'जसको', 'जसबाट', 'जसमा', 'जसलाई', 'जसले', 'जस्तै', 'जस्तो', 'जस्तोस

In [19]:
def clean_text(news):
  #remove punctuations
  sentences = news.replace("\n", " ").split(u"।")
  f = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
  f = [x.strip() for x in f if len(x)>0]
  return f
  
def remove_stopwords(sentences):
  #remove stopwords
  data  = []
  for sentence in sentences:
    words = sentence.split(" ")
    temp = [word for word in words if word not in stop_words]
    data.append(temp)

  return data

In [1]:
text = open("/content/drive/MyDrive/NLP/corpus.txt",encoding="utf8", errors='ignore').read()

In [20]:
sentences = clean_text(text)

In [21]:
data = remove_stopwords(sentences)

In [24]:
data[7]

['भनेर', 'होइन', 'जनताको', 'मन', 'जित्न', 'सकिन्छ']

## Create Word2Vec

In [25]:
from gensim.models import Word2Vec

In [26]:
model = Word2Vec(sentences=data, size=100, window=3, min_count=3, workers=4)

In [27]:
model.most_similar("समाचार")

[('समचार', 0.7439061403274536),
 ('सामाचार', 0.7239913940429688),
 ('समाचारहरू', 0.6754885911941528),
 ('खबर', 0.6719961166381836),
 ('समाचारको', 0.6371269822120667),
 ('समाचारहरु', 0.6367899179458618),
 ('समाचारका', 0.6230233907699585),
 ('पत्रपत्रिकामा', 0.6185054183006287),
 ('समाचारलाई', 0.617088794708252),
 ('पत्रिकामा', 0.612929105758667)]

In [28]:
model.most_similar("सकिन्छ")

[('सकिन्थ्यो', 0.8736274242401123),
 ('सक्छौं', 0.8586857914924622),
 ('सकिएला', 0.8556004762649536),
 ('सकियोस्', 0.8427200317382812),
 ('सकिँदैन', 0.8376457095146179),
 ('सक्छौँ', 0.8277298808097839),
 ('सकिएन', 0.8234168887138367),
 ('सकिने', 0.8220771551132202),
 ('सकिन्न', 0.8046677112579346),
 ('सक्नुहुन्छ', 0.7948466539382935)]

In [55]:
model.save("/content/drive/MyDrive/NLP/nepali_word_embeddings")

In [27]:
!du /content/drive/MyDrive/NLP/nepali_word_embeddings -h

35M	/content/drive/MyDrive/NLP/nepali_word_embeddings
